<a href="https://colab.research.google.com/github/indra622/tutorials/blob/master/CrisperWhisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install jiwer datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00


In [3]:
import os
import sys
import torch

from datasets import load_dataset
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from CrisperWhisper.utils import adjust_pauses_for_hf_pipeline_output



device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "nyrahealth/CrisperWhisper"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=2,
    #return_timestamps='word',
    torch_dtype=torch_dtype,
    device=device,
)

# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]
# hf_pipeline_output = pipe(sample)
# print(hf_pipeline_output)
#crisper_whisper_result = adjust_pauses_for_hf_pipeline_output(hf_pipeline_output)
#print(crisper_whisper_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/76.1k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/878k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/366k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [4]:
ds = load_dataset("mozilla-foundation/common_voice_16_1", "ko", split="test")
samples = ds['audio']
pipeline_output = pipe(samples, generate_kwargs={"task": "transcribe", "language": "ko"})

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_1.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

The repository for mozilla-foundation/common_voice_16_1 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_16_1.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...


Reading metadata...: 282it [00:00, 55679.22it/s]


Generating other split: 0 examples [00:00, ? examples/s]


Reading metadata...: 1832it [00:00, 58971.33it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 237it [00:00, 56893.89it/s]
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:

# pipeline_output
results = [ f['text'] for f in pipeline_output ]
print(results)

[' 이거 놔 놓으라고.', ' 선비 는 생각지 않은 이 말에 앞이 아득해지며 방안 이 핑핑 돌았다.', ' 마스크 잘 쓰고 다니고 길 조심해서 다녀와.', ' 그런지라.', ' 민 수 는 지 갑 속 에서 돈 을 내어 덕 코 앞으로 밀어 놓았 다.', ' 어리석은 잔 은 그 노를 다 드러내어도, 지혜로운 잔 은 그 노를 억제 하느니라.', ' 그 말 에 해온들은 손바닥이 뜨겁도록 박수를 한다.', ' 오 분 내로 끊어.', ' 그는 놀라 자세히 보니 그가 찾아갔던 동무 였다.', ' 절대 오제를 후회 하지 마라.', ' 네 말 은 네 행동 이 된다.', ' 그래서', ' 엘리베이터만 10 년 이에요.', ' 벽에 맞아 떨어진 돈은 다시 술 끓이는 양푼에 떨어졌다.', ' 참 경 안 앞 까지 다달아서 야, 그 는 턱 에 닿은 숨 을 돌리고, 걸음도 듯 추 잡았다.', ' chrome 대신 firefox 를 사용하여 privacy 를 지키자.', ' 저리 안 가!', ' 친절히 는 무슨 고리 타분한 냄새 를 후끈 맡으며 방으로 들어앉았다.', ' 영신이 의 저고리 는 수세미 가 들고 치마 주름 까지 주르륵 뜯어졌다.', ' 이 못 은 언제 어떻게 생겼는지 아무도 아는 사람이 없을 것 이다.', ' 그만 갤 것 이지, 어 이 이리 굳이 오 너!', ' 이 조례 의 시행에 필요한 상황은 규칙 으로 정한다.', ' 신철이는 저 햇바라를 바라보면서 지금 11시 반 이나 되었을 것을 짐작하였다.', ' 신철이는 옥점 의 이러한 대답 을 듣기 위해 부러 물었던 것 이다.', ' 불만 먹고 왜 성례 안 하지?', ' 내리실 문 은 오른쪽 입니다.', ' 오늘은, 다행히, 담배 의 꼬툴 이 남 하 있으 니, 그것 을 빨면서 발 새를 우변 내지 않았 다.', ' 내 배 속으로 나서 도 단체 그놈의 속 을 두려다 볼 수가 있어야 지.', ' 건�달폐 와 논단이들이 어우러져서 약물 이 아닌 누룩 공물 을 마시고 그 심부름 을 하는 모양 이다.', ' 돌아와 보니까 추누가 와서 자리를

In [13]:
import re
from jiwer import wer, cer, Compose, RemovePunctuation, ToLowerCase, RemoveWhiteSpace, RemoveEmptyStrings

hyp = results
ref = ds['sentence']

def preprocess_text_list(sentence_list):
  # 각 문장에서 알파벳, 숫자, 띄어쓰기를 제외한 모든 문자 제거
  return [re.sub(r'[^가-힣a-zA-Z0-9\s]', '', sentence) for sentence in sentence_list]



# 전처리된 텍스트로 WER, CER 계산
ground_truth_processed = preprocess_text_list(ref)
hypothesis_processed = preprocess_text_list(hyp)

wer_value = wer(ground_truth_processed, hypothesis_processed)
cer_value = cer(ground_truth_processed, hypothesis_processed)

In [15]:
print(ground_truth_processed)
print(hypothesis_processed)

['이거 놔 놓으라고', '선비는 생각지 않은 이 말에 앞이 아뜩해지며 방 안이 핑핑 돌았다', '마스크 잘 쓰고 다니고 길 조심해서 다녀와', '그런지라', '민수는 지갑 속에서 돈을 내어 덕호 앞으로 밀어 놓았다', '어리석은 자는 그 노를 다 드러내어도 지혜로운 자는 그 노를 억제하느니라', '그 말에 회원들은 손바닥이 뜨겁도록 박수를 한다', '오 분 내로 끊어', '그는 놀라 자세히 보니 그가 찾아가던 동무였다', '절대 어제를 후회하지 마라', '네 말은 네 행동이 된다', '그래서', '엘리베이터만 십 년이에요', '벽에 맞아 떨어진 돈은 다시 술 끓이는 양푼에 떨어졌다', '창경원 앞까지 다다라서야 그는 턱에 닿은 숨을 돌리고 걸음도 늦추잡았다', '크롬 대신 파이어폭스를 사용하여 프라이버시를 지키자', '저리 안 가', '신철이는 무슨 고리타분한 냄새를 후끈 맡으며 방으로 들어앉았다', '영신의 저고리는 수세미가 되고 치마 주름까지 주루루 트더졌다', '이 못은 언제 어떻게 생겼는지 아무도 아는 사람이 없을 것이다', '그만 갤 것이지 어이 이리 굳이 오노', '이 조례의 시행에 필요한 사항은 규칙으로 정한다', '신철이는 저 햇발을 바라보면서 지금 열한시 반이나 되었을 것을 짐작하였다', '신철이는 옥점의 이러한 대답을 듣기 위하여 부러 물었던 것이다', '부려만 먹구 왜 성례 안하지유', '내리실문은 오른쪽입니다', '오늘은 다행히 담배 꼬투리나마 있으니 그것을 빨면서 발새를 우벼 내지 않았다', '내 뱃속으루 났어두 당최 그눔의 속을 들여다볼 수가 있어야지', '건달패와 논다니들이 어우러져서 약물이 아닌 누룩 국물을 마시고 그 심부름을 하는 모양이다', '돌아와 보니까 춘우가 와서 자리를 깔고 잠이 혼곤히 들어잔다', '싱앗대는 이렇게 말하며 유서방을 곁눈질해 보았다', '아까 팽경창에게 욕을 보고 추운 겨울의 거리를 지향 없이 돌아다닐 때', '춘우는 생각하는 눈으로 옆을 흘겨보며 물었다', '학부형 중에서 한 사람이 나서며 물었다', 

In [17]:
print(wer_value, cer_value)

0.7232558139534884 0.18201204214751632


In [19]:
for h, r in zip(hypothesis_processed, ground_truth_processed):
  print(f'{h} | {r}')

 이거 놔 놓으라고 | 이거 놔 놓으라고
 선비 는 생각지 않은 이 말에 앞이 아득해지며 방안 이 핑핑 돌았다 | 선비는 생각지 않은 이 말에 앞이 아뜩해지며 방 안이 핑핑 돌았다
 마스크 잘 쓰고 다니고 길 조심해서 다녀와 | 마스크 잘 쓰고 다니고 길 조심해서 다녀와
 그런지라 | 그런지라
 민 수 는 지 갑 속 에서 돈 을 내어 덕 코 앞으로 밀어 놓았 다 | 민수는 지갑 속에서 돈을 내어 덕호 앞으로 밀어 놓았다
 어리석은 잔 은 그 노를 다 드러내어도 지혜로운 잔 은 그 노를 억제 하느니라 | 어리석은 자는 그 노를 다 드러내어도 지혜로운 자는 그 노를 억제하느니라
 그 말 에 해온들은 손바닥이 뜨겁도록 박수를 한다 | 그 말에 회원들은 손바닥이 뜨겁도록 박수를 한다
 오 분 내로 끊어 | 오 분 내로 끊어
 그는 놀라 자세히 보니 그가 찾아갔던 동무 였다 | 그는 놀라 자세히 보니 그가 찾아가던 동무였다
 절대 오제를 후회 하지 마라 | 절대 어제를 후회하지 마라
 네 말 은 네 행동 이 된다 | 네 말은 네 행동이 된다
 그래서 | 그래서
 엘리베이터만 10 년 이에요 | 엘리베이터만 십 년이에요
 벽에 맞아 떨어진 돈은 다시 술 끓이는 양푼에 떨어졌다 | 벽에 맞아 떨어진 돈은 다시 술 끓이는 양푼에 떨어졌다
 참 경 안 앞 까지 다달아서 야 그 는 턱 에 닿은 숨 을 돌리고 걸음도 듯 추 잡았다 | 창경원 앞까지 다다라서야 그는 턱에 닿은 숨을 돌리고 걸음도 늦추잡았다
 chrome 대신 firefox 를 사용하여 privacy 를 지키자 | 크롬 대신 파이어폭스를 사용하여 프라이버시를 지키자
 저리 안 가 | 저리 안 가
 친절히 는 무슨 고리 타분한 냄새 를 후끈 맡으며 방으로 들어앉았다 | 신철이는 무슨 고리타분한 냄새를 후끈 맡으며 방으로 들어앉았다
 영신이 의 저고리 는 수세미 가 들고 치마 주름 까지 주르륵 뜯어졌다 | 영신의 저고리는 수세미가 되고 치마 주름까지 주루루 트더졌다
 이 못 은 언제 어떻게 생겼는지 아무도 아는 사람이 